In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import numpy as np 
import torch
import imageio

In [2]:
img_arr = imageio.imread(r'C:\Users\ian\Downloads\bobby.jpg')
img_arr.shape

(720, 1280, 3)

In [3]:
img = torch.from_numpy(img_arr)

In [4]:
out = img.permute(2, 0, 1)

In [5]:
out.shape

torch.Size([3, 720, 1280])

In [6]:
batch_size = 100
batch = torch.zeros(batch_size, 3, 256, 256, dtype = torch.uint8)

In [7]:
batch = batch.float()
batch /= 255

In [9]:
dir_path = r'C:\Users\ian\Desktop\dlwpt-code-master\dlwpt-code-master\data\p1ch4\volumetric-dicom\2-LUNG 3.0  B70f-04083'
vol_arr = imageio.volread(dir_path, 'DICOM')
vol_arr.shape

Reading DICOM (examining files): 1/99 files (1.0%99/99 files (100.0%)
  Found 1 correct series.
Reading DICOM (loading data): 65/99  (65.799/99  (100.0%)


(99, 512, 512)

In [10]:
vol = torch.from_numpy(vol_arr).float()
vol = torch.unsqueeze(vol, 1)
vol = torch.unsqueeze(vol, 2)

In [11]:
import csv

In [14]:
wine_path = r'C:\Users\ian\Desktop\dlwpt-code-master\dlwpt-code-master\data\p1ch4\tabular-wine\winequality-white.csv'
wine_numpy = np.loadtxt(wine_path, dtype = np.float, delimiter = ';', skiprows= 1)
wine_numpy

<ipython-input-14-ff83e34147c7>:2: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  wine_numpy = np.loadtxt(wine_path, dtype = np.float, delimiter = ';', skiprows= 1)


array([[ 7.  ,  0.27,  0.36, ...,  0.45,  8.8 ,  6.  ],
       [ 6.3 ,  0.3 ,  0.34, ...,  0.49,  9.5 ,  6.  ],
       [ 8.1 ,  0.28,  0.4 , ...,  0.44, 10.1 ,  6.  ],
       ...,
       [ 6.5 ,  0.24,  0.19, ...,  0.46,  9.4 ,  6.  ],
       [ 5.5 ,  0.29,  0.3 , ...,  0.38, 12.8 ,  7.  ],
       [ 6.  ,  0.21,  0.38, ...,  0.32, 11.8 ,  6.  ]])

In [17]:
col_list = next(csv.reader(open(wine_path), delimiter = ';'))

In [18]:
wine_numpy.shape, col_list

((4898, 12),
 ['fixed acidity',
  'volatile acidity',
  'citric acid',
  'residual sugar',
  'chlorides',
  'free sulfur dioxide',
  'total sulfur dioxide',
  'density',
  'pH',
  'sulphates',
  'alcohol',
  'quality'])

In [19]:
wineq = torch.from_numpy(wine_numpy)
wineq.shape, wineq.dtype

(torch.Size([4898, 12]), torch.float64)

In [20]:
data = wineq[:, :-1]

In [21]:
data, data.shape

(tensor([[ 7.0000,  0.2700,  0.3600,  ...,  3.0000,  0.4500,  8.8000],
         [ 6.3000,  0.3000,  0.3400,  ...,  3.3000,  0.4900,  9.5000],
         [ 8.1000,  0.2800,  0.4000,  ...,  3.2600,  0.4400, 10.1000],
         ...,
         [ 6.5000,  0.2400,  0.1900,  ...,  2.9900,  0.4600,  9.4000],
         [ 5.5000,  0.2900,  0.3000,  ...,  3.3400,  0.3800, 12.8000],
         [ 6.0000,  0.2100,  0.3800,  ...,  3.2600,  0.3200, 11.8000]],
        dtype=torch.float64),
 torch.Size([4898, 11]))

In [28]:
target = wineq[:,-1]
target, target.shape

(tensor([6., 6., 6.,  ..., 6., 7., 6.], dtype=torch.float64),
 torch.Size([4898]))

In [29]:
target = target.long()

In [32]:
target_onehot = torch.zeros(target.shape[0], 10)
a = target_onehot.scatter_(1, target.unsqueeze(1), 1.0)
a, a.shape

(tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 1., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]),
 torch.Size([4898, 10]))

In [34]:
data_mean = torch.mean(data, dim = 0)

In [35]:
data_mean

tensor([6.8548e+00, 2.7824e-01, 3.3419e-01, 6.3914e+00, 4.5772e-02, 3.5308e+01,
        1.3836e+02, 9.9403e-01, 3.1883e+00, 4.8985e-01, 1.0514e+01],
       dtype=torch.float64)

In [36]:
data_var = torch.var(data, dim = 0)

In [37]:
data_var

tensor([7.1211e-01, 1.0160e-02, 1.4646e-02, 2.5726e+01, 4.7733e-04, 2.8924e+02,
        1.8061e+03, 8.9455e-06, 2.2801e-02, 1.3025e-02, 1.5144e+00],
       dtype=torch.float64)

In [38]:
data_normalize = (data - data_mean)/torch.sqrt(data_var)

In [40]:
data_normalize

tensor([[ 1.7208e-01, -8.1762e-02,  2.1326e-01,  ..., -1.2468e+00,
         -3.4915e-01, -1.3930e+00],
        [-6.5743e-01,  2.1587e-01,  4.7996e-02,  ...,  7.3995e-01,
          1.3417e-03, -8.2419e-01],
        [ 1.4756e+00,  1.7450e-02,  5.4378e-01,  ...,  4.7505e-01,
         -4.3677e-01, -3.3663e-01],
        ...,
        [-4.2043e-01, -3.7940e-01, -1.1915e+00,  ..., -1.3130e+00,
         -2.6153e-01, -9.0545e-01],
        [-1.6054e+00,  1.1666e-01, -2.8253e-01,  ...,  1.0049e+00,
         -9.6251e-01,  1.8574e+00],
        [-1.0129e+00, -6.7703e-01,  3.7852e-01,  ...,  4.7505e-01,
         -1.4882e+00,  1.0448e+00]], dtype=torch.float64)

In [41]:
bad_index = target <= 3
bad_index.shape, bad_index.dtype, bad_index.sum()

(torch.Size([4898]), torch.bool, tensor(20))

In [44]:
bike_numpy = np.loadtxt(r'C:\Users\ian\Desktop\dlwpt-code-master\dlwpt-code-master\data\p1ch4\bike-sharing-dataset\hour-fixed.csv', dtype = np.float32, delimiter = ',', skiprows = 1, converters = {1: lambda x:float(x[8:10])})

In [45]:
bikes = torch.from_numpy(bike_numpy)

In [46]:
bikes

tensor([[1.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 3.0000e+00, 1.3000e+01,
         1.6000e+01],
        [2.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 8.0000e+00, 3.2000e+01,
         4.0000e+01],
        [3.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 5.0000e+00, 2.7000e+01,
         3.2000e+01],
        ...,
        [1.7377e+04, 3.1000e+01, 1.0000e+00,  ..., 7.0000e+00, 8.3000e+01,
         9.0000e+01],
        [1.7378e+04, 3.1000e+01, 1.0000e+00,  ..., 1.3000e+01, 4.8000e+01,
         6.1000e+01],
        [1.7379e+04, 3.1000e+01, 1.0000e+00,  ..., 1.2000e+01, 3.7000e+01,
         4.9000e+01]])

In [47]:
daily_bikes = bikes.view(-1, 24, bikes.shape[1])

In [49]:
daily_bikes.shape

torch.Size([730, 24, 17])

In [50]:
daily_bikes = daily_bikes.transpose(1, 2)

In [51]:
 first_day = bikes[:24].long()

In [53]:
weather_onehot = torch.zeros(24, 4)

In [54]:
first_day[:, 9]

tensor([1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 2, 2, 2, 2])

In [56]:
weather_onehot.scatter_(1, first_day[:, 9].unsqueeze(1).long()-1, 1)

tensor([[1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.]])

In [59]:
torch.cat((bikes[:24], weather_onehot), 1)[:1]

tensor([[ 1.0000,  1.0000,  1.0000,  0.0000,  1.0000,  0.0000,  0.0000,  6.0000,
          0.0000,  1.0000,  0.2400,  0.2879,  0.8100,  0.0000,  3.0000, 13.0000,
         16.0000,  1.0000,  0.0000,  0.0000,  0.0000]])

In [61]:
daily_weather_onehot = torch.zeros(daily_bikes.shape[0], 4, daily_bikes.shape[2])

In [64]:
daily_weather_onehot.scatter_(1, daily_bikes[:, 9, :].long().unsqueeze(1)-1, 1.0)

tensor([[[1., 1., 1.,  ..., 0., 0., 0.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [0., 0., 0.,  ..., 1., 1., 1.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[1., 1., 1.,  ..., 1., 1., 1.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0

In [66]:
daily_bikes = torch.cat((daily_bikes, daily_weather_onehot), 1)

tensor([[1., 1., 1.,  ..., 2., 2., 2.],
        [2., 2., 2.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        ...,
        [2., 2., 2.,  ..., 2., 2., 2.],
        [2., 2., 2.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.]])

In [2]:
with open(r'C:\Users\ian\Desktop\dlwpt-code-master\dlwpt-code-master\data\p1ch4\jane-austen\1342-0.txt', encoding = 'utf8') as f:
    text = f.read()

In [3]:
lines = text.split('\n')
line = lines[200]

In [4]:
line

'“Impossible, Mr. Bennet, impossible, when I am not acquainted with him'

In [5]:
letter_t = torch.zeros(len(line), 128)

In [6]:
letter_t.shape

torch.Size([70, 128])

In [8]:
for i, letter in enumerate(line.lower().strip()):
    letter_index = ord(letter) if ord(letter) < 128 else 0
    letter_t[i][letter_index] = 1

In [10]:
letter_t.shape

torch.Size([70, 128])

In [15]:
def clean_words(input_str):
    punctuation = '.,;:"!?_-“'
    word_list = input_str.lower().split()
    word_list = [word.strip(punctuation) for word in word_list]
    return word_list
word_in_line = clean_words(line)
line, word_in_line

('“Impossible, Mr. Bennet, impossible, when I am not acquainted with him',
 ['impossible',
  'mr',
  'bennet',
  'impossible',
  'when',
  'i',
  'am',
  'not',
  'acquainted',
  'with',
  'him'])

In [20]:
word_list = sorted(set(clean_words(text)))
word2index_dict = {word: i for (i, word) in enumerate(word_list)}
len(word2index_dict), word2index_dict['impossible']

(8267, 3848)